In [1]:
%gui qt
import os
import numpy as np

from pathlib import Path
import h5py, tifffile
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

from image_alignment import align_images as ai

import astra

# ***1. read data***

In [2]:
data_center_path = "/media/Disk2/data/JingShi_2020_Sept/data_center"

'''
dpath = '/media/Disk2/data/JingShi_2020_Sept/raw'
refn = os.path.join(dpath, 'PbIBrCsRb_xrf_tomo_cr.tif_pcc_TRANSLATION.h5_sr_TRANSLATION.h5')
tgtn = os.path.join(dpath, 'PbIBrCsRb_xrf_tomo_i.tif')
itr = 99
'''

basename = 'PbIBrCsRb_xrf_tomo_cr.tif'
#basename = 'PbIBrCsRb_xrf_tomo_cr.tif_pcc_TRANSLATION.h5'
dpath = '/media/Disk2/data/JingShi_2020_Sept/raw/'
dfn = Path(os.path.join(dpath, f'{basename}'))
ofn = Path(f'/media/Disk2/data/JingShi_2020_Sept/raw/{basename}_combo_alignment.h5')

ref = np.zeros([200, 62, 256])
if basename.split('.')[-1] == 'h5':
    with h5py.File(dfn, 'r') as f:
        #imgs[:, :34, 28:228] = f['/proj_corr/corrected_proj/iter_00_shifted_images'][:, :34, :]
        #imgs[:, 38:66, 28:228] = f['/proj_corr/corrected_proj/iter_00_shifted_images'][:, 38:66, :]
        ref[:] = f['/proj_corr/corrected_proj/iter_00_shifted_images'][:]
elif basename.split('.')[-1] == 'tif':
    ref[:, :62, 28:228] = np.swapaxes(tifffile.imread(dfn), 0, 1)[:]
    #ref[:, 38:66, 28:228] = ref[:, 34:62, 28:228] 
    #ref[:, 34:38, 28:228] = 0

angs = np.loadtxt(os.path.join(dpath, 'angle_list.txt'))*np.pi/180
#angs = np.linspace(74, -58, num=66, endpoint=True)*np.pi/180

# ***2. align images***

In [7]:
aier = ai(data_order='astra')
aier.set_data(ref, angs)
aier.set_rec_alg('astra_cgls', **{})
aier.data_order = 'astra'
aier.corr_cfg = {0:{'cor_name':'pc', 'sub_itr':5, 'cfg':{'mask_thres':1e-2}},
                 1:{'cor_name':'sr', 'sub_itr':15, 'cfg':{'mode':'RIGID_BODY'}}}
aier.proj_cfg = {'prj_name':'astra', 'cfg':{'proj_geom':None, 'vol_geom':None}}
aier.align_stack(1)

0 0 0
    astra recons finishes at Sat Sep 19 01:07:10 2020
    registration starts at Sat Sep 19 01:07:10 2020
0


/home/xiao/software/anaconda3/ws4_backup/user_packages/image_alignment.py:100: RuntimeWarning: invalid value encountered in greater
  cfg['reference_mask'] = (ref>thres)
/home/xiao/software/anaconda3/ws4_backup/user_packages/image_alignment.py:100: RuntimeWarning: invalid value encountered in greater
  cfg['reference_mask'] = (ref>thres)
/home/xiao/software/anaconda3/ws4_backup/user_packages/image_alignment.py:100: RuntimeWarning: invalid value encountered in greater
  cfg['reference_mask'] = (ref>thres)
/home/xiao/software/anaconda3/ws4_backup/user_packages/image_alignment.py:100: RuntimeWarning: invalid value encountered in greater
  cfg['reference_mask'] = (ref>thres)
/home/xiao/software/anaconda3/ws4_backup/user_packages/image_alignment.py:100: RuntimeWarning: invalid value encountered in greater
  cfg['reference_mask'] = (ref>thres)
/home/xiao/software/anaconda3/ws4_backup/user_packages/image_alignment.py:100: RuntimeWarning: invalid value encountered in greater
  cfg['reference_m

1
2
    registration finishes at Sat Sep 19 01:07:18 2020
0 0 1
    astra recons finishes at Sat Sep 19 01:07:19 2020
    registration starts at Sat Sep 19 01:07:19 2020
0
1
2
    registration finishes at Sat Sep 19 01:07:26 2020
0 0 2
    astra recons finishes at Sat Sep 19 01:07:27 2020
    registration starts at Sat Sep 19 01:07:28 2020
0
1
2
    registration finishes at Sat Sep 19 01:07:35 2020
0 0 3
    astra recons finishes at Sat Sep 19 01:07:36 2020
    registration starts at Sat Sep 19 01:07:36 2020
0
1
2
    registration finishes at Sat Sep 19 01:07:44 2020
0 0 4
    astra recons finishes at Sat Sep 19 01:07:45 2020
    registration starts at Sat Sep 19 01:07:45 2020
0
1
2
    registration finishes at Sat Sep 19 01:07:54 2020
0 1 0
    astra recons finishes at Sat Sep 19 01:07:55 2020
    registration starts at Sat Sep 19 01:07:55 2020
0
1
2
    registration finishes at Sat Sep 19 01:08:00 2020
0 1 1
    astra recons finishes at Sat Sep 19 01:08:01 2020
    registration start

In [9]:
def disp(x):
    plt.imshow(aier.mov_data[:, x, :])
widgets.interact(disp, x=(0, 61, 1))

interactive(children=(IntSlider(value=30, description='x', max=61), Output()), _dom_classes=('widget-interact'…

<function __main__.disp(x)>

In [ ]:
if 'astra' in aier.rec_alg['alg_name']:
    if aier.data_order == 'astra':
        vol_geom = astra.creators.create_vol_geom(aier.data_dim[2], aier.data_dim[2], aier.data_dim[0])

In [ ]:
print(aier.mov_data)

In [ ]:
from skimage.registration import phase_cross_correlation

In [ ]:
help(phase_cross_correlation)